In [ ]:
import glob 
import os 
import numpy as np
import pandas as pd
from utils import *

## Settings and Paramters

In [4]:
CPU_LIMITS= [ '250m', '500m', '750m', '1000m', '1250m', '1500m', '1750m', '2000m' ]
MEMORY_LIMITS=[ '128Mi', '256Mi', '512Mi', '768Mi', '1024Mi' ]
instance_types = ['m5', 'm5a', 'c5', 'c5a', 'c6g', 'm6g']
benchmarks = {"transcode": "cats-1", "pigo-faceblur": "faces2", "pigo-face-detector": "faces2", "classifier": "bridge-8k-1920x1200", "linpack": "5000", "s3": "cats-1", "openfaas-ocr": "doc"}
calculate_cpu_mem_costs()
# Weights of execution time and execution cost objectives
weights = [(0.0, 1.0), (0.25, 0.75), (0.50, 0.50), (0.75, 0.25), (1.0, 0.0)]
thresholds =  [0.05, 0.1, 0.2]

## Potential alternative configurations
The following script gets the alternative configuration choices with different instance types that provides performance (as defined by the weighted objective function) within the thresholds mentioned above. 

In [5]:
for benchmark in benchmarks.keys():
    data_name = benchmarks[benchmark]
    print(benchmark)
    runtimes, costs, configs = get_all_runtimes_costs_from_data(instance_types, CPU_LIMITS, MEMORY_LIMITS, benchmark, data_name, return_configs=True)
    
    for weight in weights:
        print(weight)
        dataframe = pd.DataFrame(columns = ['metric', 'memory', 'cpu'])

        runtime_weight = weight[0]
        cost_weight = weight[1]

        minimum_runtime, minimum_cost, _, _ = get_best_configuration(instance_types, CPU_LIMITS, MEMORY_LIMITS, benchmark, data_name, metric=str(runtime_weight)+'/'+str(cost_weight))

        for i in range(0, len(runtimes)):
            config = configs[i]
            runtime = runtimes[i]
            cost = costs[i]

            mem = config["memory"]
            cpu = config["cpu"]
            instance_type = config["instance_type"]

            metric_value =  ((runtime/minimum_runtime) * runtime_weight) + ((cost/minimum_cost) * cost_weight)

            dataframe = dataframe.append({'metric': metric_value, 'memory': mem, 
                                            'cpu': instance_type+ '_'+ cpu}, ignore_index=True)

        dataframe.sort_values(by=["metric"], inplace=True)
        best_config = dataframe.iloc[0]
        # print("Best configuration")
        print(best_config)
        
        for threshold in thresholds:
            count = 0 
            alternatives = []
            for index, row in dataframe.iterrows():
                if  (row["metric"]<= (1+threshold)*best_config["metric"]) and (row["cpu"].split("_")[0] != best_config["cpu"].split("_")[0]) and (row["cpu"].split("_")[0] not in alternatives):
                    alternatives.append(row["cpu"].split("_")[0])
                    count += 1
                    # print(row)
            print(alternatives)
            print("Number of instance types that are within " + str(threshold*100) + "% of the best configuration: " + str(count) + "\n\n")


transcode
(0.0, 1.0)
metric           1
memory         512
cpu       m6g_2000
Name: 237, dtype: object
['c6g']
Number of instance types that are within 5.0% of the best configuration: 1


['c6g', 'c5a']
Number of instance types that are within 10.0% of the best configuration: 2


['c6g', 'c5a']
Number of instance types that are within 20.0% of the best configuration: 2


(0.25, 0.75)
metric           1
memory         512
cpu       m6g_2000
Name: 237, dtype: object
['c5a', 'c6g']
Number of instance types that are within 5.0% of the best configuration: 2


['c5a', 'c6g']
Number of instance types that are within 10.0% of the best configuration: 2


['c5a', 'c6g']
Number of instance types that are within 20.0% of the best configuration: 2


(0.5, 0.5)
metric           1
memory         512
cpu       c5a_2000
Name: 157, dtype: object
['m6g', 'c6g']
Number of instance types that are within 5.0% of the best configuration: 2


['m6g', 'c6g']
Number of instance types that are within 10.0% of the